In [11]:
import mysql.connector as sql
import glob
import pandas as pd
from datetime import datetime

In [12]:
df = pd.read_csv("/home/garage/workspace/price_update_crontab/upbit_krweth_1min.csv")

In [13]:
df = df.tail(11)
print(df)

                timestamp      open     close      high       low      volume
1608  2020-09-18T09:52:00  452500.0  452550.0  452550.0  452450.0   45.179840
1609  2020-09-18T09:53:00  452500.0  452600.0  452600.0  452500.0   99.012877
1610  2020-09-18T09:54:00  452600.0  452600.0  452600.0  452500.0   16.087115
1611  2020-09-18T09:55:00  452500.0  452500.0  452600.0  452500.0   10.277513
1612  2020-09-18T09:56:00  452500.0  452400.0  452500.0  452400.0    3.256670
1613  2020-09-18T09:57:00  452400.0  452400.0  452600.0  452400.0   17.407238
1614  2020-09-18T09:58:00  452500.0  452400.0  452500.0  452400.0    5.562064
1615  2020-09-18T09:59:00  452400.0  452600.0  452600.0  452350.0  125.788384
1616  2020-09-18T10:00:00  452600.0  452400.0  452650.0  452400.0   12.784606
1617  2020-09-18T10:01:00  452400.0  453000.0  453050.0  452400.0   15.323395
1618  2020-09-18T10:02:00  453050.0  453000.0  453200.0  453000.0    5.202440


In [14]:
db_connection = sql.connect(host='root.cqyptexqvznx.ap-northeast-2.rds.amazonaws.com',port=int(3306), database='garage_test', user='root', password='koscom!234')
db_cursor = db_connection.cursor()

In [15]:

query = 'SELECT max(base_time) FROM history_prc_upbit WHERE base_dt = %s and coin_type = %s'
db_cursor.execute(query,(datetime.today().strftime("%Y%m%d"),'eth',))
print(datetime.today().strftime("%Y%m%d"))
max_time = db_cursor.fetchone()[0]
print(max_time)


20200918
10:02:00


In [16]:
for i in range(len(df)):
    tmp = str(df.iloc[i]['timestamp']).replace("-","")
    tmp = tmp.split('T')
    base_dt = tmp[0]
    
    coin_type = 'eth'
    base_time = tmp[1]
    if max_time >= tmp[1] :
        continue  
    open_price = df.iloc[i]['open']
    close_price = df.iloc[i]['close']
    high_price = df.iloc[i]['high']
    low_price = df.iloc[i]['low']
    volumn = df.iloc[i]['volume']

    query = "INSERT INTO history_prc_upbit VALUES('{}','{}','{}',{},{},{},{},{})".format(base_dt,base_time,coin_type,open_price,close_price,high_price,low_price,volumn)
    db_cursor.execute(query)
    print(query)
    db_connection.commit()
db_connection.close()